# RAG 체감해보기

In [ ]:
import os

from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

MY_OPENAI_KEY = os.getenv("MY_OPENAI_KEY")

client = OpenAI(api_key=MY_OPENAI_KEY)

prompt = '내일 날씨는 어때?'

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

죄송합니다. 날씨 정보는 실시간으로 업데이트되는 정보이기 때문에 정확한 내일 날씨를 알려드릴 수 없습니다. 근처 기상청 웹사이트나 기상 앱을 통해 내일 날씨를 확인해보시는 것을 추천드립니다.


### 어떻게 실시간으로 부족한 데이터를 주입할까?

- Prompt에 부족한 데이터를 추가하는 방식

In [4]:
# 기상청 단기예보
weather_info = """(종합) 오늘 낮까지 비 곳, 오늘 강풍과 풍랑 유의, 내일 늦은 오후~저녁 경기동부 소나기, 모레 낮부터 비
○ (오늘, 30일) 대체로 흐림, 서울.경기도(경기남서부 제외) 중심 낮(12~15시)까지 비 곳
○ (내일, 7월 1일) 구름많음, 대기불안정으로 늦은 오후(15~18시)부터 저녁(18~21시) 사이 경기동부 소나기 곳, 서해5도 대체로 맑다가 저녁부터 구름많아짐
○ (모레, 7월 2일) 대체로 흐림, 서울.인천.경기도 낮(12~15시)부터 비, 서해5도(연평도 부근) 늦은 밤(21~24시) 가끔 비 곳
* 예상 강수량(30일 낮까지)
- 서울.경기도(경기남서부 제외): 5mm 미만
* 소나기에 의한 예상 강수량(7월 1일 늦은 오후~저녁)
- 경기동부: 5~20mm
* 예상 강수량(7월 2일 낮~)
- 서울.인천.경기도: 10~50mm
- 서해5도(7월 2일 밤): 5~20mm"""

prompt = '내일 날씨는 어때?' + '\n\n' + weather_info

completion = client.chat.completions.create(
    model='gpt-3.5-turbo-0125',
    messages=[{'role': 'user', 'content': prompt}],
    temperature=0.0
)

print(completion.choices[0].message.content)

내일은 구름많고 대기불안정으로 늦은 오후부터 저녁 사이에 경기동부에서 소나기가 올 것으로 예상됩니다. 모레는 대체로 흐리고 서울, 인천, 경기도에서 낮부터 비가 오겠습니다. 서해5도는 늦은 밤에 가끔 비가 올 것으로 예상됩니다. 오늘은 대체로 흐리고 서울, 경기도 중심으로 낮까지 비가 오는 곳이 있을 것으로 보입니다. 강풍과 풍랑에 유의하시기 바랍니다.


### 어떻게 질문과 연관된 정보를 가져올까? (RAG의 핵심)

- RAG 에서 Retrieval 이 나오는 이유
  - Retrieval = 연관 텍스트 가져오는 행위 (검색도 Retrieval에 포함)

- 어떻게 Retrieve 할까?
  - 방법 1) 텍스트에서 특정 단어가 자주 발생 --> 해당 단어랑 연관이 있을꺼다!
  - 방법 2) 근데 자주 발생하느 단어가 항상 자주 발생하는 단어 --> 이런건 가중치를 줄이자
  - 방법 3) 딥러닝 모델한테 텍스트 간 문서를 직접 학습시키면 어떨까?
    - 2012년 Word2Vec 논문에서 시작. 흔히 임베딩이라고 한다.

#### 가장 간단한 방법 시연 1

In [6]:
def count_common_words(text1, text2):
    words1 = text1.split()
    words2 = text2.split()
    common_words = set(words1) & set(words2)

    return len(common_words)

question = '한국의 전통 음식에 대해 무엇을 알고 계신가요?'

ref_texts = [
    '한국의 전통 음식은 다양한 재료와 조리법으로 유명합니다. 김치, 불고기, 비빔밥 등이 대표적인 예시입니다.',
    '한국의 음식 문화는 건강에 좋은 재료를 사용하는 것으로 알려져 있습니다. 발요 식품인 김치가 대표적입니다.',
    '한국의 역사와 문화는 매우 흥미롭습니다. 고궁과 한복, 그리고 태권도 등이 유명한 문화 요소입니다.'
]

common_word_counts = [count_common_words(question, ref_text) for ref_text in ref_texts]

most_similar_index = common_word_counts.index(max(common_word_counts))
print(f'가장 유사한 텍스트 번호: {most_similar_index + 1}')
print(f'겹치는 단어의 수: {common_word_counts[most_similar_index]}')

가장 유사한 텍스트 번호: 1
겹치는 단어의 수: 2


### 토큰 임베딩

In [7]:
client = OpenAI(api_key=MY_OPENAI_KEY)

embedding = client.embeddings.create(
    input='내일 날씨는 어때?',
    model='text-embedding-3-small'
)

embedding

CreateEmbeddingResponse(data=[Embedding(embedding=[-0.020439069718122482, 0.005229348316788673, -0.009943580254912376, -0.0018029062775895, 0.03644446283578873, -0.0059836250729858875, 0.037640269845724106, 0.014478441327810287, -0.0337584987282753, 0.00029463949613273144, -0.012160418555140495, 0.03903844207525253, -0.0645366832613945, -0.04544059559702873, -0.0007008108077570796, -0.014496838673949242, -0.0654197409749031, 0.03000551275908947, 0.015968598425388336, 0.014506036415696144, -0.014975160360336304, -0.030962156131863594, -0.01598699577152729, 0.030207879841327667, -0.029821541160345078, 0.012997482903301716, -0.038744088262319565, 0.02286747470498085, 0.002849235897883773, -0.016603294759988785, -0.012114426121115685, -0.03107253834605217, 0.0177439097315073, -0.021432507783174515, 0.031771622598171234, 0.011525722220540047, 0.028276193886995316, 0.029600778594613075, -0.04069416970014572, -0.01461641862988472, -0.01847059093415737, -0.008149872533977032, -0.01163610443472

In [8]:
embedding.data[0].embedding

[-0.020439069718122482,
 0.005229348316788673,
 -0.009943580254912376,
 -0.0018029062775895,
 0.03644446283578873,
 -0.0059836250729858875,
 0.037640269845724106,
 0.014478441327810287,
 -0.0337584987282753,
 0.00029463949613273144,
 -0.012160418555140495,
 0.03903844207525253,
 -0.0645366832613945,
 -0.04544059559702873,
 -0.0007008108077570796,
 -0.014496838673949242,
 -0.0654197409749031,
 0.03000551275908947,
 0.015968598425388336,
 0.014506036415696144,
 -0.014975160360336304,
 -0.030962156131863594,
 -0.01598699577152729,
 0.030207879841327667,
 -0.029821541160345078,
 0.012997482903301716,
 -0.038744088262319565,
 0.02286747470498085,
 0.002849235897883773,
 -0.016603294759988785,
 -0.012114426121115685,
 -0.03107253834605217,
 0.0177439097315073,
 -0.021432507783174515,
 0.031771622598171234,
 0.011525722220540047,
 0.028276193886995316,
 0.029600778594613075,
 -0.04069416970014572,
 -0.01461641862988472,
 -0.01847059093415737,
 -0.008149872533977032,
 -0.011636104434728622,
 -

In [9]:
len(embedding.data[0].embedding)

1536

In [10]:
embedding.usage

Usage(prompt_tokens=13, total_tokens=13)

### Embeddings API 를 활용해 텍스트 간 유사도 구하기

- 두 텍스트 간 얼마나 비슷한 지는 Cosine Similarity 사용
- -1~1 사이 표현 1은 완전히 같고, 0은 관련없음, -1은 정반대
- Cosine Similarity 계산은 numpy 연산 아주 빠르게 해주는 라이브러리 사용

In [11]:
import numpy as np

def get_embedding(text, model='text-embedding-3-small'):
    client = OpenAI(api_key=MY_OPENAI_KEY)
    response = client.embeddings.create(
        input=text,
        model=model
    )
    
    return response.data[0].embedding


def cosine_similarity(a, b):

    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


text1 = "오늘은 날씨가 좋아서 공원에 산책을 갔습니다."
text2 = "날씨가 맑아서 야외 활동하기 좋은 날이었습니다."

embedding1 = get_embedding(text1)
embedding2 = get_embedding(text2)

similarity = cosine_similarity(embedding1, embedding2)

print(f"두 텍스트의 유사도: {similarity}")

두 텍스트의 유사도: 0.41610426330959066


### OpenAI Embeddings API를 활용해서 관련 있는 텍스트를 Retrieve하는 예시

In [ ]:
text1 = """(종합) 오늘 낮까지 비 곳, 오늘 강풍과 풍랑 유의, 내일 늦은 오후~저녁 경기동부 소나기, 모레 낮부터 비
○ (오늘, 30일) 대체로 흐림, 서울.경기도(경기남서부 제외) 중심 낮(12~15시)까지 비 곳
○ (내일, 7월 1일) 구름많음, 대기불안정으로 늦은 오후(15~18시)부터 저녁(18~21시) 사이 경기동부 소나기 곳, 서해5도 대체로 맑다가 저녁부터 구름많아짐
○ (모레, 7월 2일) 대체로 흐림, 서울.인천.경기도 낮(12~15시)부터 비, 서해5도(연평도 부근) 늦은 밤(21~24시) 가끔 비 곳
* 예상 강수량(30일 낮까지)
- 서울.경기도(경기남서부 제외): 5mm 미만
* 소나기에 의한 예상 강수량(7월 1일 늦은 오후~저녁)
- 경기동부: 5~20mm
* 예상 강수량(7월 2일 낮~)
- 서울.인천.경기도: 10~50mm
- 서해5도(7월 2일 밤): 5~20mm"""

text2 = """서울의 주요 관광지 안내:
서울은 한국의 수도로서 다양한 문화와 역사를 자랑하는 도시입니다. 다음은 서울에서 꼭 가봐야 할 주요 관광지입니다.

경복궁: 조선 시대의 대표적인 궁궐로, 한국 전통 건축의 아름다움을 느낄 수 있습니다.
남산타워: 서울의 전경을 한눈에 볼 수 있는 전망대입니다. 케이블카를 타고 올라가면 더욱 즐거운 경험이 됩니다.
인사동: 전통 공예품과 예술 작품을 구경할 수 있는 거리로, 다양한 전통 음식점과 카페도 즐길 수 있습니다.
홍대: 젊음의 거리로, 다양한 카페, 레스토랑, 그리고 스트리트 공연을 즐길 수 있습니다.
강남: 현대적이고 세련된 분위기의 지역으로, 쇼핑과 미식 탐험에 최적의 장소입니다.
서울을 여행하기 좋은 시기는 봄과 가을입니다. 날씨가 쾌적하고, 다양한 축제와 이벤트가 열리기 때문입니다. 여행을 계획할 때는 서울의 대중교통 시스템을 잘 활용하면 편리하게 이동할 수 있습니다.
"""

question = '내일 날씨는 어때?'

In [ ]:
embedding_question = get_embedding(question)
embedding_text1 = get_embedding(text1)
embedding_text2 = get_embedding(text2)

similarity1 = cosine_similarity(embedding_question, embedding_text1)
similarity2 = cosine_similarity(embedding_question, embedding_text2)

similarity1, similarity2

In [ ]:
embedding_question = get_embedding(question, model='text-embedding-3-large')
embedding_text1 = get_embedding(text1, model='text-embedding-3-large')
embedding_text2 = get_embedding(text2, model='text-embedding-3-large')

similarity1 = cosine_similarity(embedding_question, embedding_text1)
similarity2 = cosine_similarity(embedding_question, embedding_text2)

similarity1, similarity2

### RAG 데이터 확보
- Allganize의 RAG 데이터 중 금융 도메인 데이터 200개 활용

In [2]:
# https://huggingface.co/datasets/allganize/rag-ko

from huggingface_hub import snapshot_download

snapshot_download(
    repo_id='allganize/rag-ko',
    repo_type='dataset',
    local_dir='./res/rag-ko',
    local_dir_use_symlinks=False
)

print()

Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 119.95it/s]

#### 데이터 설명
- 질문과 정답이 있고, 답변에 필요한 정보(Context)가 질문 별로 3개씩 존재
  - 3개의 정보 중 하나가 실제로 필요한 정보이고, 나머지 2개는 필요하지 않은 정보
- Column 별 설명
  - system: 시스템 프롬프트 (Context 3개가 여기에 있음)
  - humman: 질문
  - answer_context_summary: 실제로 필요한 Context
  - answer_position: 3개의 Context 중 몇 번째가 실제로 필요한 Context인지
  - answer: 정답

In [3]:
import pandas as pd

df = pd.read_parquet('./res/rag-ko/data/test-00000-of-00001.parquet')
df

,index,system,human,answer,answer_position,answer_context_title,answer_context_summary
0,0,You are a financial expert.\nYou are fluent at...,글로벌 저금리 현상이 부각된 원인은 무엇인가요?,"글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...",3,132579145651725164_KIFRT2021-02 (2),2. 뉴노멀의 도래(신 3저의 도래)\n최근 코로나 사태를 포함하여 2008년 글로...
1,1,You are a financial expert.\nYou are fluent at...,고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.,고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...,2,132579145651725164_KIFRT2021-02 (2),축소되어 자산운용의 어려움이 가중\n∙ 투자자 입장에서는 원하는 수익률을 얻기 어려...
2,2,You are a financial expert.\nYou are fluent at...,20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...,20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...,1,132579145651725164_KIFRT2021-02 (2),1. 디지털화의 진전과 금융투자업의 변화 : 20세기 이후\n가. 1950~60년대...
3,3,You are a financial expert.\nYou are fluent at...,2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?,"2000년대 이후 글로벌 IB들은 스스로 전산화를 추진하였으며, 이와 밀접하게 연관...",3,132579145651725164_KIFRT2021-02 (2),"∙ 또한, 투자은행의 명성에 의존하는 대표적인 업무인 IPO, M&A\n등도 전산화..."
4,4,You are a financial expert.\nYou are fluent at...,금융시스템 개혁법은 언제 제정되었나요?,금융시스템 개혁법은 1998년에 제정되었습니다.,3,132579145651725164_KIFRT2021-02 (2),및 일본 내 해외투자 수요를 포용하는데 적극적\n일련의 변화과정에서 일본 금융당국의...
...,...,...,...,...,...,...,...
195,195,You are a financial expert.\nYou are fluent at...,우리나라 은행들이 기업대출을 통해 외부자금을 필요로 하는 산업에 어떻게 기여하였나요?,우리나라 은행들은 기업대출을 통해 외부자금을 필요로 하는 산업에 자금을 효율적으로 ...,3,132859249008797508_FARPT21-06,... 긍정적인 측면과 부정적인 측면\n∙ 우리나라 은행의 기업대출이 사업체 수 증...
196,196,You are a financial expert.\nYou are fluent at...,은행의 기업대출이 중소기업의 부가가치 성장에 어떻게 기여하였는지 설명해주세요.,은행의 기업대출이 중소기업의 부가가치 성장에 기여하는 방식은 크게 두 가지로 볼 수...,2,132859249008797508_FARPT21-06,... 데\n은행의 기업대출 대부분도 중소기업대출이어서 과연 은행이 중소기업대출을\...
197,197,You are a financial expert.\nYou are fluent at...,기업 저축의 역설에 대해 설명해주세요.,"기업 저축의 역설이란, 기업들이 지나치게 많은 자금을 저축하면서 경제의 역동성이 상...",3,130328682579485000_KIFFS2013-08,... 현재의 기업 저축률은 불필요하게 높은 수준일 가능성을 시사\n4. 2004년...
198,198,You are a financial expert.\nYou are fluent at...,명목 경제 성장률이 높은 것이 경기호황을 의미하는 이유는 무엇인가요?,명목 경제 성장률이 높다는 것은 경기가 호황인 상태를 의미합니다. 이는 경제가 활발...,1,130328682579485000_KIFFS2013-08,... 비해 명목 성장률이 특별히 높았던 기간이 아니었다. 50) 먼저 2001년...


In [4]:
print(df.iloc[0]['system'])

You are a financial expert.
You are fluent at Korean.
Answer the question to User in CONTEXT.
When using information from the CONTEXT, You should always cite the context ID number "number" using "(ID: number)" format.

CONTEXT="""
(context 1)= Title: 132579145651725164_KIFRT2021-02 (2).pdf
... 변화의 동인은 크게 두 가지로 분류할
수 있다. 하나는 금융업계 전반에 부각되는 디지털화(digitalization)1)이고
다른 하나는 저금리·저성장으로 대표되는 뉴노멀(new normal)이다.  ... . 뉴노멀의 특징으로는 금융투자
및 자산운용의 어려움이 가중되는 경향, 고수익-고위험 부문으로의 쏠림
현상 등의 위험요인과 함께 자산관리서비스 수요의 증가 등 대응 노력을
살펴본다. 제III장은 디지털화 및 뉴노멀에 대응하는 해외 사례들을 살펴본다.  ... 뉴노멀의
주요 특징인 저성장, 저금리 등은 일본의 금융투자업계가 30년 전부터 고민해
온 문제이기 때문이다.  ... 제III장에서 살펴본 해외
사례들에서 살펴본 금융투자업계 변화의 규칙성이 어느 정도 유지될 것이라는
1 디지털화(digitalization)에 대한 ...

(context 2)= Title: 132599894715831637_KIFFR20201-03 (2).pdf
3. 문제점
1) 기업신용위험 평가항목의 적정성
글로벌 금융위기 이후 저금리 기조가 지속되었고, 최근 추가적으로
금리가 하락함에 따라 이자보상배율 등의 지표들이 기업의 부실을
측정하는 데 적정한 지표인지에 대한 검토가 필요
기업신용위험 정 ... 작용할 것으로 예상
저금리 기조가 지속되는 가운데 금융비용이 감소하여 이자보상배
율이 기업의 부실을 가늠하는 지표로 적합한지에 대해 의문이
발생
더불어 코로나19 

In [5]:
print(df.iloc[0]['answer_context_summary'])

2. 뉴노멀의 도래(신 3저의 도래)
최근 코로나 사태를 포함하여 2008년 글로벌 금융위기 이후 전세계적인
저금리·저성장 상황
∙ 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의
완화적 통화정책으로 글로벌 저금리 현상이 부각
∙ 코로나 팬데믹이 진정되더라도 글로벌 저금리 추세가 지속되는 가운
데 우리나라의 저금리 기조도 장기화될 가능성이 높음. 이러한 추세는 글로벌 금융위기 이후의 전세계적인 완화적 통화정책과
그 이전부터 이어진 경제구조적 요인이 혼재된 결과
∙ 선진국의 성장세가 둔화되고, 신흥국이 성장하면서 글로벌 무역 불
균형(global imbalance)이 심화되고 저축과잉현상(global savings
glut) 발생
∙ 기 ... 가.  ...


#### 전처리 코드

In [6]:
import re

def extract_contexts(system_prompt):
    context_match = re.search(r'CONTEXT="""\n(.*?)"""', system_prompt, re.DOTALL)
    if not context_match:
        return []

    context_text = context_match.group(1)

    contexts = re.findall(r'\(context \d+\)=(.*?)(?=\n\(context \d+\)=|\Z)', context_text, re.DOTALL)

    cleaned_contexts = []
    for context in contexts:
        lines = context.split('\n')
        cleaned_lines = [line for line in lines if not line.strip().startswith('Title:')]
        cleaned_context = '\n'.join(cleaned_lines).strip()
        cleaned_contexts.append(cleaned_context)

    return cleaned_contexts

system_prompt = df.iloc[0]['system']

extracted_contexts = extract_contexts(system_prompt)

for i, context in enumerate(extracted_contexts, 1):
    print(f"Context {i}:")
    print(context)
    print("-" * 50)

Context 1:
... 변화의 동인은 크게 두 가지로 분류할
수 있다. 하나는 금융업계 전반에 부각되는 디지털화(digitalization)1)이고
다른 하나는 저금리·저성장으로 대표되는 뉴노멀(new normal)이다.  ... . 뉴노멀의 특징으로는 금융투자
및 자산운용의 어려움이 가중되는 경향, 고수익-고위험 부문으로의 쏠림
현상 등의 위험요인과 함께 자산관리서비스 수요의 증가 등 대응 노력을
살펴본다. 제III장은 디지털화 및 뉴노멀에 대응하는 해외 사례들을 살펴본다.  ... 뉴노멀의
주요 특징인 저성장, 저금리 등은 일본의 금융투자업계가 30년 전부터 고민해
온 문제이기 때문이다.  ... 제III장에서 살펴본 해외
사례들에서 살펴본 금융투자업계 변화의 규칙성이 어느 정도 유지될 것이라는
1 디지털화(digitalization)에 대한 ...
--------------------------------------------------
Context 2:
3. 문제점
1) 기업신용위험 평가항목의 적정성
글로벌 금융위기 이후 저금리 기조가 지속되었고, 최근 추가적으로
금리가 하락함에 따라 이자보상배율 등의 지표들이 기업의 부실을
측정하는 데 적정한 지표인지에 대한 검토가 필요
기업신용위험 정 ... 작용할 것으로 예상
저금리 기조가 지속되는 가운데 금융비용이 감소하여 이자보상배
율이 기업의 부실을 가늠하는 지표로 적합한지에 대해 의문이
발생
더불어 코로나19 사태로 인한 중소기업에 대한 금융권 대출 만기
연장 및 이자상환 유예조치로 이 ... 는 추세
이러한 졸업비율 감소 현상의 원인으로 저조한 경기의 영향이
있었던 것으로 추정되지만 개시연도를 기준으로 졸 ...
--------------------------------------------------
Context 3:
2. 뉴노멀의 도래(신 3저의 도래)
최근 코로나 사태를 포함하여 2008년 글로벌 금융위기 이후 전세계적인
저금리·저성장 상황
∙ 글로벌 금융위기를 극복하는 과정에서 미국, 유럽,

In [7]:
extracted_contexts[2] == df.iloc[0]['answer_context_summary']

True

In [8]:
contexts = []
for i in range(len(df)):
    system_prompt = df.iloc[i]['system']
    extracted_contexts = extract_contexts(system_prompt)
    contexts.append(extracted_contexts)

In [9]:
questions = [df.iloc[i]['human'] for i in range(len(df))]
contexts_answers_idxs = [df.iloc[i]['answer_position'] - 1 for i in range(len(df))]
contexts_answers = [df.iloc[i]['answer_context_summary'] for i in range(len(df))]
answers = [df.iloc[i]['answer'] for i in range(len(df))]

In [10]:
import pickle

rag_data = {
    'questions': questions,
    'contexts': contexts,
    'contexts_answer_idx': contexts_answers_idxs,
    'contexts_answers': contexts_answers,
    'answers': answers
}

with open('./res/rag_data.pkl', 'wb') as f:
    pickle.dump(rag_data, f)

# RAG 성능 평가 기법

### RAG 평가
- 크게는 2가지 평가 방법론
  - (1) LLM + RAG의 end-to-end 성능 평가
    - 이전 파트에서 여러 번 진행 (ex. LLM-as-a-judge...)
  - (2) RAG 로직의 성능 평가
    - 2번에 집중 예정

In [18]:
import pickle

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

In [19]:
rag_data['questions'][0]

'글로벌 저금리 현상이 부각된 원인은 무엇인가요?'

In [20]:
rag_data['contexts'][0]

['... 변화의 동인은 크게 두 가지로 분류할\n수 있다. 하나는 금융업계 전반에 부각되는 디지털화(digitalization)1)이고\n다른 하나는 저금리·저성장으로 대표되는 뉴노멀(new normal)이다.  ... . 뉴노멀의 특징으로는 금융투자\n및 자산운용의 어려움이 가중되는 경향, 고수익-고위험 부문으로의 쏠림\n현상 등의 위험요인과 함께 자산관리서비스 수요의 증가 등 대응 노력을\n살펴본다. 제III장은 디지털화 및 뉴노멀에 대응하는 해외 사례들을 살펴본다.  ... 뉴노멀의\n주요 특징인 저성장, 저금리 등은 일본의 금융투자업계가 30년 전부터 고민해\n온 문제이기 때문이다.  ... 제III장에서 살펴본 해외\n사례들에서 살펴본 금융투자업계 변화의 규칙성이 어느 정도 유지될 것이라는\n1 디지털화(digitalization)에 대한 ...',
 '3. 문제점\n1) 기업신용위험 평가항목의 적정성\n글로벌 금융위기 이후 저금리 기조가 지속되었고, 최근 추가적으로\n금리가 하락함에 따라 이자보상배율 등의 지표들이 기업의 부실을\n측정하는 데 적정한 지표인지에 대한 검토가 필요\n기업신용위험 정 ... 작용할 것으로 예상\n저금리 기조가 지속되는 가운데 금융비용이 감소하여 이자보상배\n율이 기업의 부실을 가늠하는 지표로 적합한지에 대해 의문이\n발생\n더불어 코로나19 사태로 인한 중소기업에 대한 금융권 대출 만기\n연장 및 이자상환 유예조치로 이 ... 는 추세\n이러한 졸업비율 감소 현상의 원인으로 저조한 경기의 영향이\n있었던 것으로 추정되지만 개시연도를 기준으로 졸 ...',
 '2. 뉴노멀의 도래(신 3저의 도래)\n최근 코로나 사태를 포함하여 2008년 글로벌 금융위기 이후 전세계적인\n저금리·저성장 상황\n∙ 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의\n완화적 통화정책으로 글로벌 저금리 현상이 부각\n∙ 코로나 팬데믹이 진정되더라도 글로벌 저금리 추세가 지속되는 가운\n데 우리나라의 저금리 기조도 장기화될 가능

#### Accuracy
- 정답이 있는 상황이라 가장 정확한 평가
- Reference 또는 정답이 있을 때만 사용 가능
  - 실제로는 없는 경우가 대부분

In [ ]:
from utils import get_embedding, cosine_similarity

embed_q = get_embedding(rag_data['questions'][0])
embed_c0 = get_embedding(rag_data['contexts'][0][0])
embed_c1 = get_embedding(rag_data['contexts'][0][1])
embed_c2 = get_embedding(rag_data['contexts'][0][2])

In [22]:
print(cosine_similarity(embed_q, embed_c0))
print(cosine_similarity(embed_q, embed_c1))
print(cosine_similarity(embed_q, embed_c2))

0.36210903416669354
0.44104521085802917
0.41554819344807353


In [23]:
rag_data['contexts_answer_idx'][0]

np.int64(2)

In [24]:
embed_q = get_embedding(rag_data['questions'][0], model='text-embedding-3-large')
embed_c0 = get_embedding(rag_data['contexts'][0][0], model='text-embedding-3-large')
embed_c1 = get_embedding(rag_data['contexts'][0][1], model='text-embedding-3-large')
embed_c2 = get_embedding(rag_data['contexts'][0][2], model='text-embedding-3-large')

print(cosine_similarity(embed_q, embed_c0))
print(cosine_similarity(embed_q, embed_c1))
print(cosine_similarity(embed_q, embed_c2))

0.40704031995543827
0.39666413635201253
0.5327138694846036


In [25]:
from tqdm import tqdm

num_questions = 10
num_contexts = 3

top_context_indices = []

for i in tqdm(range(num_questions)):
    embed_q = get_embedding(rag_data['questions'][i])

    similarities = []
    for j in range(num_contexts):
        embed_c = get_embedding(rag_data['contexts'][i][j])
        similarity = cosine_similarity(embed_q, embed_c)
        similarities.append(similarity)

    top_context_index = similarities.index(max(similarities))
    top_context_indices.append(top_context_index)

print(f"Top Context Indices: {top_context_indices}")

100%|██████████| 10/10 [00:28<00:00,  2.82s/it]

Top Context Indices: [1, 1, 0, 1, 1, 1, 2, 0, 1, 1]


In [26]:
rag_data['contexts_answer_idx'][:num_questions]

[np.int64(2),
 np.int64(1),
 np.int64(0),
 np.int64(2),
 np.int64(2),
 np.int64(1),
 np.int64(2),
 np.int64(2),
 np.int64(1),
 np.int64(1)]

In [27]:
def calculate_accuracy(predicted, actual):
    correct = sum(p == a for p, a in zip(predicted, actual))
    total = len(predicted)
    
    accuracy = correct / total
    return accuracy

accuracy = calculate_accuracy(top_context_indices, rag_data['contexts_answer_idx'][:10])
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 60.00%


### 정답 Context가 없는 경우
- RAGAS
  - GPT-4를 활용한 RAG 로직 평가용 라이브러리
    - RAG 뿐만 아니라 LLM 단독 그리고 LLM + RAG 평가도 가능
  - RAG 평가의 경우 정답 Context가 있어도 사용이 가능하지만 정답이 없는 경우에도 사용이 가능
  - OpenAI에서도 RAG 로직 평가를 위해 사용 (OpenAI Dev Day)
  - 기본으로 제공하는 평가용 System Prompt가 영문 기반이라 한글 평가는 상대적으로 정확도가 약간 떨어지는 편

In [28]:
contexts_predictions = []
for i in range(len(top_context_indices)):
    index = top_context_indices[i]
    contexts_predictions.append([rag_data['contexts'][i][index]])
contexts_predictions

[['3. 문제점\n1) 기업신용위험 평가항목의 적정성\n글로벌 금융위기 이후 저금리 기조가 지속되었고, 최근 추가적으로\n금리가 하락함에 따라 이자보상배율 등의 지표들이 기업의 부실을\n측정하는 데 적정한 지표인지에 대한 검토가 필요\n기업신용위험 정 ... 작용할 것으로 예상\n저금리 기조가 지속되는 가운데 금융비용이 감소하여 이자보상배\n율이 기업의 부실을 가늠하는 지표로 적합한지에 대해 의문이\n발생\n더불어 코로나19 사태로 인한 중소기업에 대한 금융권 대출 만기\n연장 및 이자상환 유예조치로 이 ... 는 추세\n이러한 졸업비율 감소 현상의 원인으로 저조한 경기의 영향이\n있었던 것으로 추정되지만 개시연도를 기준으로 졸 ...'],
 ['축소되어 자산운용의 어려움이 가중\n∙ 투자자 입장에서는 원하는 수익률을 얻기 어려워 보다 높은 수익률을\n추구하는 투자행태 ... 나. 고수익-고위험 부문으로 쏠림현상\n금융회사, 가계 등의 수익률 추구 성향(search for yield)이 강화되어\n위험자산으로 과도한 자금 유입이 발생할 수 있음.\n∙ 저금리 기조 하에서는 투자자의 고수익 추구로 인해 고위험 파생상\n품, 부동산금융 등 리스크가 큰 분야로의 쏠림현상 심화 가능성\n∙ 사모펀드 부문의 빠른 성장, 파생상품 자산의 급증 등 개인의 투자위\n험도 증가하는 추세\n* 고수익-고위험 추구  ...  등)으로 기대수익률을 충족시키지\n못해 고위험-고수익을 추구할 가능성 증가\n∙ 개인 투자자들은 이에 대한 전문성 부족으로  ...'],
 ['1. 디지털화의 진전과 금융투자업의 변화 : 20세기 이후\n가. 1950~60년대\nMorrison and Wilhelm(2007), Jensen(1993) 등은 컴퓨터 기술의\n발전과 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 큰\n영향을 끼치는 것은 20세기 중반 이후로 평가\n1960년대 후반 소위 백오피스 위기를 거치면서 백오피스 업무, 거래소\n거래 중심의 전산화가 가속 ... 1980년대 이후 

- Context Recall
  - RAG 모델이 가져온 Context 문장에서 질문의 정답 문장이 얼마나 많이 존재하는 지 확인
  - 실제 질문의 답변과 Context 정보만 사용 (정답 Context X)

In [31]:
from datasets import Dataset 
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_relevancy, context_recall, context_precision

data_samples = {
    'question': rag_data['questions'][:num_questions],
    'contexts' : contexts_predictions,
    'ground_truth': rag_data['answers'][:num_questions]
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[context_recall])
score

ImportError: cannot import name 'context_relevancy' from 'ragas.metrics' (d:\myProjects\ai-deep-learning\part8_rag\.venv\Lib\site-packages\ragas\metrics\__init__.py)